In [23]:
!pip install requests beautifulsoup4 pandas
!pip install atlasclient



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'atlasclient' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'atlasclient'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for atlasclient: filename=atlasclient-1.0.0-py3-none-any.whl size=29174 sha256=ab13d3076d4e94f0a62a8bdca566f4c49c4b5ce199520a6b4bbe41b2b901c421
  Stored in directory: /Users/vaibhavpandey/Library/Caches/pip/wheels/b2/77/c6/94d6fd4ca781735e546114b844f6f5a25d592664fd305e9096
Successfully built atlasclient

[notice] A new release of pip is available: 25.2 -> 25.3
[noti

In [22]:
import pandas as pd

df_srs = pd.read_csv("download_samples_tsv-2.tsv", sep="\t", dtype=str)
df_srs = df_srs.rename(columns={"#sid": "SRS"})
df_srs["SRS"] = df_srs["SRS"].astype(str)

df_meta = pd.read_csv("metadata.csv", dtype=str)

print("SRS columns:", df_srs.columns.tolist())
print("Meta columns:", df_meta.columns.tolist())

SRS columns: ['SRS', 'name', 'note', 'sample_env', 'keywords_clean', 'taxa_stats', 'num_rids', 'num_hq_runs', 'rids', 'projects', 'publications']
Meta columns: ['subjectID', 'SampleID', 'age_at_collection', 'collection_month', 'delivery', 'gest_time', 'gender', 'country', 'Exclusive_breast_feeding', 'Breast_feeding_end', 'Regular_formula', 'hydrosylated_formula', 'partly_hydrosylated_formula', 'Any_baby_formula', 'Fruits_and_berries', 'Corn', 'Rice', 'Wheat', 'Oat', 'Barley', 'Rye', 'Cereal', 'Other_grains', 'Root_vegetables', 'Vegetables', 'Eggs', 'Soy', 'Milk', 'Meat', 'Fish', 'Other_food', 'Other_than_BF', 'bf_length', 'num_abx_treatments', 'num_abx_first_year', 'abx_first_year', 'after_abx', 'num_preceeding_abx', 'hla_risk_class', 'seroconverted', 'num_aabs', 'totalige', 'totalige_log', 'allergy_milk', 'allergy_egg', 'allergy_peanut', 'allergy_dustmite', 'totalige_high', 'allergy_cat', 'allergy_dog', 'allergy_birch', 'allergy_timothy', 'gid_wgs', 'mgx_reads', 'mgx_pool', 'mgx_reads

In [23]:
import requests
import re

def get_host_subject_id(srs_id: str) -> str | None:
    """
    Use ENA browser XML API to get host_subject_id for a given SRS accession.
    This matches the 'Original MetaData' you see in MicrobeAtlas.
    """
    url = f"https://www.ebi.ac.uk/ena/browser/api/xml/{srs_id}"
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
    except Exception as e:
        print(f"[ERROR] SRS={srs_id}: {e}")
        return None

    xml = r.text

    # ENA attribute structure is like:
    # <SAMPLE_ATTRIBUTE>
    #   <TAG>host_subject_id</TAG>
    #   <VALUE>P018832</VALUE>
    # </SAMPLE_ATTRIBUTE>
    m = re.search(
        r"<TAG>\s*host_subject_id\s*</TAG>\s*<VALUE>\s*([^<\s]+)\s*</VALUE>",
        xml,
        flags=re.IGNORECASE,
    )
    if m:
        return m.group(1)

    print(f"[WARN] no host_subject_id found in XML for {srs_id}")
    return None

In [24]:
test_srs = "SRS1719087"
print("Testing SRS:", test_srs)
print("host_subject_id:", get_host_subject_id(test_srs))

Testing SRS: SRS1719087
host_subject_id: P018832


In [25]:
import pandas as pd
import time

df_srs = pd.read_csv("download_samples_tsv-2.tsv", sep="\t", dtype=str)
df_srs = df_srs.rename(columns={"#sid": "SRS"})
df_srs["SRS"] = df_srs["SRS"].astype(str)

unique_srs = df_srs["SRS"].dropna().unique().tolist()
print("Number of unique SRS:", len(unique_srs))

rows = []
for i, srs in enumerate(unique_srs, start=1):
    subj = get_host_subject_id(srs)
    rows.append({"SRS": srs, "host_subject_id": subj})

    if i % 50 == 0:
        print(f"{i}/{len(unique_srs)} SRS processed")
        time.sleep(0.2)

df_map = pd.DataFrame(rows)
df_map.to_csv("srs_to_host_subject_id.csv", index=False)
df_map.head()

Number of unique SRS: 785
50/785 SRS processed
100/785 SRS processed
150/785 SRS processed
200/785 SRS processed
250/785 SRS processed
300/785 SRS processed
350/785 SRS processed
400/785 SRS processed
450/785 SRS processed
500/785 SRS processed
550/785 SRS processed
600/785 SRS processed
650/785 SRS processed
700/785 SRS processed
750/785 SRS processed


,SRS,host_subject_id
0,SRS1719087,P018832
1,SRS1719088,P017743
2,SRS1719089,P000648
3,SRS1719090,T022883
4,SRS1719091,T012374


In [ ]:
# CELL 5 REMOVED: Old broken merge (was merging only on subjectID, creating Cartesian product)
# The correct merge is done in Cell 7 using both subjectID AND collection_month

Detected subject column: subjectID
Merged columns: ['SRS', 'subjectID', 'SampleID', 'age_at_collection', 'collection_month', 'delivery', 'gest_time', 'gender', 'country', 'Exclusive_breast_feeding', 'Breast_feeding_end', 'Regular_formula', 'hydrosylated_formula', 'partly_hydrosylated_formula', 'Any_baby_formula', 'Fruits_and_berries', 'Corn', 'Rice', 'Wheat', 'Oat', 'Barley', 'Rye', 'Cereal', 'Other_grains', 'Root_vegetables', 'Vegetables', 'Eggs', 'Soy', 'Milk', 'Meat', 'Fish', 'Other_food', 'Other_than_BF', 'bf_length', 'num_abx_treatments', 'num_abx_first_year', 'abx_first_year', 'after_abx', 'num_preceeding_abx', 'hla_risk_class', 'seroconverted', 'num_aabs', 'totalige', 'totalige_log', 'allergy_milk', 'allergy_egg', 'allergy_peanut', 'allergy_dustmite', 'totalige_high', 'allergy_cat', 'allergy_dog', 'allergy_birch', 'allergy_timothy', 'gid_wgs', 'mgx_reads', 'mgx_pool', 'mgx_reads_filtered', 'read_count_16S', 'sequencing_PDO_16S', 'gid_16s']


,SRS,subjectID,SampleID,age_at_collection,collection_month,delivery,gest_time,gender,country,Exclusive_breast_feeding,...,allergy_dog,allergy_birch,allergy_timothy,gid_wgs,mgx_reads,mgx_pool,mgx_reads_filtered,read_count_16S,sequencing_PDO_16S,gid_16s
0,SRS1719087,P018832,3104160,597.0,20.0,vaginal,278.0,Female,RUS,False,...,False,False,False,G80333,25397552.0,Plate 8,25.322641,20683.0,PDO-4156,G73770
1,SRS1719087,P018832,3100410,292.0,10.0,vaginal,278.0,Female,RUS,False,...,False,False,False,G78751,26226378.0,Plate 2,26.156407,19873.0,PDO-3146,G65936
2,SRS1719087,P018832,3100412,358.0,12.0,vaginal,278.0,Female,RUS,False,...,False,False,False,G69201,12784726.0,old_batch,12.777236,46515.0,PDO-4098,G65938
3,SRS1719087,P018832,3104039,445.0,15.0,vaginal,278.0,Female,RUS,False,...,False,False,False,NaN,NaN,NaN,NaN,101199.0,PDO-4156,G73640
4,SRS1719087,P018832,3104038,416.0,14.0,vaginal,278.0,Female,RUS,False,...,False,False,False,G78566,26477490.0,Plate 3,26.366903,32674.0,PDO-3108,G64586


In [27]:
import os

srs_month_csv = "srs_to_collection_month.csv"

if os.path.exists(srs_month_csv):
    print("="*70)
    print(f"{srs_month_csv} already exists. Loading instead of extracting...")
    df_srs_month = pd.read_csv(srs_month_csv, dtype=str)
    print(f"✓ Loaded {len(df_srs_month)} SRS samples from {srs_month_csv}")
    print(df_srs_month.head(10))
else:
    # SOLUTION: Extract host_age for all SRS samples and convert to collection_month
    print("="*70)
    print("EXTRACTING host_age FOR ALL SRS SAMPLES")
    print("="*70)

    # Extract for all unique SRS
    unique_srs = df_map['SRS'].dropna().unique().tolist()
    print(f"\nProcessing {len(unique_srs)} SRS samples...")

    srs_age_data = []
    failed_srs = []

    for i, srs in enumerate(unique_srs, start=1):
        metadata = get_sra_metadata(srs)
        
        if metadata and 'host_age' in metadata:
            try:
                age_days = int(metadata['host_age'])
                # Convert days to months (using 30.44 days/month average)
                age_months = round(age_days / 30.44)
                
                srs_age_data.append({
                    'SRS': srs,
                    'host_age_days': age_days,
                    'collection_month': age_months,
                    'host_subject_id_ena': metadata.get('host_subject_id', None)
                })
            except ValueError:
                print(f"  [{i}] {srs}: Invalid host_age value: {metadata['host_age']}")
                failed_srs.append(srs)
        else:
            print(f"  [{i}] {srs}: No host_age found")
            failed_srs.append(srs)
        
        # Progress indicator
        if i % 50 == 0:
            print(f"  Processed {i}/{len(unique_srs)} samples...")
            time.sleep(0.5)  # Be nice to the API

    print(f"\n✓ Successfully extracted {len(srs_age_data)} samples")
    print(f"✗ Failed for {len(failed_srs)} samples")

    # Create DataFrame with SRS → collection_month mapping
    df_srs_month = pd.DataFrame(srs_age_data)
    df_srs_month.to_csv(srs_month_csv, index=False)
    print(f"\nSaved to: {srs_month_csv}")
    print(df_srs_month.head(10))

srs_to_collection_month.csv already exists. Loading instead of extracting...
✓ Loaded 785 SRS samples from srs_to_collection_month.csv
          SRS host_age_days collection_month host_subject_id_ena
0  SRS1719087           686               23             P018832
1  SRS1719088           173                6             P017743
2  SRS1719089           493               16             P000648
3  SRS1719090           229                8             T022883
4  SRS1719091           502               16             T012374
5  SRS1719092           390               13             T016811
6  SRS1719093           427               14             T017394
7  SRS1719094           587               19             T007750
8  SRS1719095           598               20             T003950
9  SRS1719096           594               20             T004341


In [28]:
# CORRECT MERGE: Use SRS → collection_month mapping
print("="*70)
print("PERFORMING CORRECT MERGE WITH COLLECTION MONTH")
print("="*70)

# Load the extracted SRS → collection_month mapping
df_srs_month = pd.read_csv("srs_to_collection_month.csv", dtype=str)
df_srs_month['collection_month'] = pd.to_numeric(df_srs_month['collection_month'], errors='coerce').astype('Int64')

# Load metadata
df_meta = pd.read_csv("metadata.csv", dtype=str)
df_meta['collection_month'] = pd.to_numeric(df_meta['collection_month'], errors='coerce').astype('Int64')

# Detect subject column in metadata
subj_col = None
for c in df_meta.columns:
    if c.lower() in ("subjectid", "subject_id", "host_subject_id", "host_subjectid"):
        subj_col = c
        break

if subj_col:
    df_meta = df_meta.rename(columns={subj_col: "subjectID"})
    df_srs_month = df_srs_month.rename(columns={"host_subject_id_ena": "subjectID"})

print(f"✓ Loaded {len(df_srs_month)} SRS samples with collection months")
print(f"✓ Loaded {len(df_meta)} metadata rows")

# CORRECT MERGE: on BOTH subjectID AND collection_month
print("\nPerforming merge on ['subjectID', 'collection_month']...")
df_merged_correct = df_srs_month.merge(
    df_meta, 
    on=['subjectID', 'collection_month'], 
    how='left'
)

print(f"\n✓ Merge complete: {len(df_merged_correct)} rows")
print(f"  (Should be same as number of SRS samples: {len(df_srs_month)})")

# Check for any duplicates
duplicate_check = df_merged_correct.groupby('SRS').size()
duplicates = duplicate_check[duplicate_check > 1]

if len(duplicates) > 0:
    print(f"\n⚠ WARNING: {len(duplicates)} SRS samples still have duplicates after merge!")
    print("First few duplicates:")
    print(duplicates.head(10))
else:
    print("\n✓ SUCCESS: No sample appears in multiple rows!")

# Show sample of merged data
print("\nSample merged data:")
print(df_merged_correct[['SRS', 'subjectID', 'collection_month', 'allergy_milk', 'allergy_egg', 'allergy_peanut']].head(10))

PERFORMING CORRECT MERGE WITH COLLECTION MONTH
✓ Loaded 785 SRS samples with collection months
✓ Loaded 1946 metadata rows

Performing merge on ['subjectID', 'collection_month']...

✓ Merge complete: 826 rows
  (Should be same as number of SRS samples: 785)

⚠ WARNING: 40 SRS samples still have duplicates after merge!
First few duplicates:
SRS
SRS1719147    2
SRS1719148    2
SRS1719152    3
SRS1719166    2
SRS1719199    2
SRS1719227    2
SRS1719237    2
SRS1719283    2
SRS1719304    2
SRS1719316    2
dtype: int64

Sample merged data:
          SRS subjectID  collection_month allergy_milk allergy_egg  \
0  SRS1719087   P018832                23        False       False   
1  SRS1719088   P017743                 6          NaN         NaN   
2  SRS1719089   P000648                16        False       False   
3  SRS1719090   T022883                 8         True        True   
4  SRS1719091   T012374                16        False       False   
5  SRS1719092   T016811                1

In [29]:
# Investigate and resolve duplicates
print("="*70)
print("INVESTIGATING DUPLICATES")
print("="*70)

# Find which SRS have duplicates
duplicate_srs = df_merged_correct.groupby('SRS').size()
duplicate_srs = duplicate_srs[duplicate_srs > 1].index.tolist()

print(f"\n{len(duplicate_srs)} SRS samples have duplicates")
print("\nExample: SRS1719152 (has 3 rows)")
example = df_merged_correct[df_merged_correct['SRS'] == 'SRS1719152'][['SRS', 'subjectID', 'collection_month', 'SampleID', 'allergy_milk', 'allergy_egg', 'allergy_peanut']]
print(example)

print("\nREASON: Some patients have multiple physical samples (different SampleIDs) collected in the same month.")
print("SOLUTION: Aggregate allergy data - if ANY sample in that month shows allergy, mark as allergic.")

# Aggregate: for each SRS, take max of allergy values (True > False)
# This ensures if any sample that month was allergic, we mark it as allergic

def aggregate_allergy_data(group):
    """For duplicate rows, aggregate allergy info"""
    result = group.iloc[0].copy()  # Start with first row
    
    # For allergy columns, take the max (True > False)
    allergy_cols = ['allergy_milk', 'allergy_egg', 'allergy_peanut', 
                    'allergy_dustmite', 'totalige_high', 'allergy_cat', 
                    'allergy_dog', 'allergy_birch', 'allergy_timothy']
    
    for col in allergy_cols:
        if col in group.columns:
            # Convert to boolean, taking True if any row is True
            vals = group[col].fillna('False').astype(str).str.lower()
            result[col] = 'True' if any(v in {'1', 'true', 'yes'} for v in vals) else 'False'
    
    return result

print("\nAggregating duplicate rows...")
df_final = df_merged_correct.groupby('SRS', as_index=False).apply(aggregate_allergy_data)
df_final = df_final.reset_index(drop=True)

print(f"✓ After aggregation: {len(df_final)} rows (one per SRS)")
print(f"✓ No duplicates: {df_final['SRS'].nunique() == len(df_final)}")

print("\nFinal data sample:")
print(df_final[['SRS', 'subjectID', 'collection_month', 'allergy_milk', 'allergy_egg', 'allergy_peanut']].head(10))

INVESTIGATING DUPLICATES

40 SRS samples have duplicates

Example: SRS1719152 (has 3 rows)
           SRS subjectID  collection_month SampleID allergy_milk allergy_egg  \
67  SRS1719152   P020604                13  3104057          NaN         NaN   
68  SRS1719152   P020604                13  3104056          NaN         NaN   
69  SRS1719152   P020604                13  3104053          NaN         NaN   

   allergy_peanut  
67            NaN  
68            NaN  
69            NaN  

REASON: Some patients have multiple physical samples (different SampleIDs) collected in the same month.
SOLUTION: Aggregate allergy data - if ANY sample in that month shows allergy, mark as allergic.

Aggregating duplicate rows...
✓ After aggregation: 785 rows (one per SRS)
✓ No duplicates: True

Final data sample:
          SRS subjectID  collection_month allergy_milk allergy_egg  \
0  SRS1719087   P018832                23        False       False   
1  SRS1719088   P017743                 6        F

/var/folders/t4/bjm8mgw92d53x5w4s4hbgbww0000gn/T/ipykernel_21840/2607847769.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final = df_merged_correct.groupby('SRS', as_index=False).apply(aggregate_allergy_data)


In [ ]:
# LONGITUDINAL LABELING: Option 3 - Latest/Maximum allergy status per patient
print("="*70)
print("APPLYING LONGITUDINAL LABELING STRATEGY")
print("="*70)

# Food allergy columns
food_allergy_cols = [
    "allergy_milk",
    "allergy_egg",
    "allergy_peanut",
]

def is_allergic(val):
    """Check if a value represents allergic status"""
    v = str(val).strip().lower()
    return v in {"1", "true", "yes"}

# For each patient, compute their MAXIMUM/LATEST allergy status across all timepoints
print("\nStep 1: Computing maximum allergy status per patient...")

patient_max_allergy = df_final.groupby('subjectID').agg({
    'allergy_milk': lambda x: any(is_allergic(v) for v in x),
    'allergy_egg': lambda x: any(is_allergic(v) for v in x),
    'allergy_peanut': lambda x: any(is_allergic(v) for v in x)
}).reset_index()

print(f"✓ Computed max allergy status for {len(patient_max_allergy)} patients")
print("\nSample patient max allergy status:")
print(patient_max_allergy.head(10))

# Apply longitudinal labels: all samples from a patient get labeled with their max allergy
print("\nStep 2: Applying longitudinal labels to all samples...")

df_longitudinal = df_final.drop(columns=food_allergy_cols).merge(
    patient_max_allergy, 
    on='subjectID', 
    how='left'
)

# Create binary label and allergen class
def get_food_allergy_label(row):
    """Binary label: 1 if any food allergy, 0 otherwise"""
    return 1 if any([row['allergy_milk'], row['allergy_egg'], row['allergy_peanut']]) else 0

def get_allergen_class(row):
    """
    Allergen class label:
    0 = non-allergic
    1 = milk allergy only
    2 = egg allergy only
    3 = peanut allergy only
    4 = multiple food allergies
    """
    allergies = []
    if row['allergy_milk']:
        allergies.append(1)
    if row['allergy_egg']:
        allergies.append(2)
    if row['allergy_peanut']:
        allergies.append(3)
    
    if len(allergies) == 0:
        return 0
    elif len(allergies) == 1:
        return allergies[0]
    else:
        return 4

df_longitudinal["label"] = df_longitudinal.apply(get_food_allergy_label, axis=1)
df_longitudinal["allergen_class"] = df_longitudinal.apply(get_allergen_class, axis=1)

print(f"✓ Labeled {len(df_longitudinal)} samples")
print("\nSample labeled data:")
print(df_longitudinal[['SRS', 'subjectID', 'collection_month', 'allergy_milk', 'allergy_egg', 'allergy_peanut', 'label', 'allergen_class']].head(20))

,SRS,subjectID,collection_month,label,allergen_class
0,SRS1719087,P018832,20.0,0,0
1,SRS1719087,P018832,10.0,0,0
2,SRS1719087,P018832,12.0,0,0
3,SRS1719087,P018832,15.0,0,0
4,SRS1719087,P018832,14.0,0,0


In [ ]:
# Verify longitudinal labeling
print("="*70)
print("VERIFICATION: Checking longitudinal labeling")
print("="*70)

# Example: Check a patient who develops allergy later
# Find a patient with milk allergy
allergic_patient = df_longitudinal[df_longitudinal['allergy_milk'] == True]['subjectID'].iloc[0] if any(df_longitudinal['allergy_milk'] == True) else None

if allergic_patient:
    print(f"\nExample: Patient {allergic_patient} (develops milk allergy)")
    patient_samples = df_longitudinal[df_longitudinal['subjectID'] == allergic_patient][
        ['SRS', 'collection_month', 'allergy_milk', 'allergy_egg', 'allergy_peanut', 'label']
    ].sort_values('collection_month')
    print(patient_samples)
    print("\n✓ All samples from this patient are labeled as allergic (label=1)")
else:
    print("\nNo allergic patients found in dataset")

# Summary statistics
print("\n" + "="*70)
print("SUMMARY STATISTICS")
print("="*70)
print(f"Total samples: {len(df_longitudinal)}")
print(f"Unique patients: {df_longitudinal['subjectID'].nunique()}")
print(f"Collection months range: {df_longitudinal['collection_month'].min()} - {df_longitudinal['collection_month'].max()}")
print(f"\nLabel distribution:")
print(df_longitudinal['label'].value_counts().sort_index())
print(f"\nAllergen class distribution:")
print(df_longitudinal['allergen_class'].value_counts().sort_index())

,SRS,subjectID,collection_month,label,allergen_class
0,SRS1719087,P018832,6,0,0
1,SRS1719087,P018832,7,0,0
2,SRS1719087,P018832,10,0,0
3,SRS1719087,P018832,11,0,0
4,SRS1719087,P018832,12,0,0


In [ ]:
# Save the longitudinally-labeled dataset
output_file = "diabimmune_longitudinal_labels.csv"
df_longitudinal.to_csv(output_file, index=False)

print("="*70)
print(f"✓ Saved longitudinally-labeled dataset to: {output_file}")
print("="*70)
print(f"Columns: {df_longitudinal.columns.tolist()}")
print(f"\nFirst 10 rows:")
print(df_longitudinal[['SRS', 'subjectID', 'collection_month', 'label', 'allergen_class']].head(10))

Master labeled file written: food_allergies_srs_labels_all_months.csv
Columns: ['SRS', 'subjectID', 'collection_month', 'label', 'allergen_class']
          SRS subjectID  collection_month  label  allergen_class
0  SRS1719087   P018832                 6      0               0
1  SRS1719087   P018832                 7      0               0
2  SRS1719087   P018832                10      0               0
3  SRS1719087   P018832                11      0               0
4  SRS1719087   P018832                12      0               0


In [ ]:
# Optional: Create per-month files for downstream analysis
import os

output_dir = "preprocessed_diabimmune_longitudinal"
os.makedirs(output_dir, exist_ok=True)

print("="*70)
print(f"Creating per-month files in: {output_dir}/")
print("="*70)

# Sanity check: Each SRS should appear in exactly ONE month
sample_month_counts = df_longitudinal.groupby("SRS")["collection_month"].nunique()
leaked_samples = sample_month_counts[sample_month_counts > 1]

if len(leaked_samples) > 0:
    print(f"⚠ WARNING: {len(leaked_samples)} samples appear in multiple months!")
    print("This should not happen with correct preprocessing!")
else:
    print(f"✓ All {len(df_longitudinal)} samples appear in exactly one month (no leakage)\n")

# Write per-month files
for month, grp in df_longitudinal.groupby("collection_month"):
    out = grp[["SRS", "subjectID", "label", "allergen_class"]].copy()
    out = out.rename(columns={"SRS": "sid", "subjectID": "patient_id"})
    
    fname = f"Month_{int(month)}.csv"
    fpath = os.path.join(output_dir, fname)
    out.to_csv(fpath, index=False)
    
    # Count labels
    allergic_count = (out['label'] == 1).sum()
    non_allergic_count = (out['label'] == 0).sum()
    print(f"Month {int(month):2d}: {len(out):3d} samples ({allergic_count} allergic, {non_allergic_count} non-allergic)")

print("\n✓ Per-month files created successfully!")

Before deduplication: 8188 rows
After deduplication: 1856 rows
Removed 6332 duplicate patient entries within same months
Max samples per patient per month after dedup: 1 (should be 1)

=== Cross-Month Sample Leak Check ===
Sample IDs that appear in multiple months:
  SRS1719087: appears in 18 months -> [6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
  SRS1719088: appears in 6 months -> [6, 7, 8, 10, 11, 12]
  SRS1719089: appears in 17 months -> [5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 24]
  SRS1719090: appears in 9 months -> [2, 3, 4, 8, 10, 13, 16, 19, 22]
  SRS1719091: appears in 8 months -> [1, 4, 7, 10, 13, 16, 20, 22]
  SRS1719092: appears in 10 months -> [1, 4, 5, 7, 13, 16, 20, 22, 28, 36]
  SRS1719093: appears in 10 months -> [2, 4, 7, 10, 12, 14, 16, 22, 28, 36]
  SRS1719094: appears in 6 months -> [1, 4, 8, 10, 12, 19]
  SRS1719095: appears in 9 months -> [1, 4, 7, 10, 13, 16, 20, 22, 36]
  SRS1719096: appears in 11 months -> [1, 4, 